# Using 🤗 PEFT & bitsandbytes to finetune a LoRa checkpoint




In [1]:
!pip install -q bitsandbytes datasets accelerate loralib
!pip install -q git+https://github.com/huggingface/transformers.git@main git+https://github.com/huggingface/peft.git

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.6/92.6 MB 11.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 35.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 265.7/265.7 kB 24.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 15.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 18.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [2]:
!nvidia-smi -L

GPU 0: Tesla T4 (UUID: GPU-13054219-2f22-af93-d9be-97fc9fa31db0)


### Setup the model

In [3]:
import os
os.environ["CUDA_VISIBLE_DEVICES"]="0"
import torch
import torch.nn as nn
import bitsandbytes as bnb
from transformers import AutoTokenizer, AutoConfig, AutoModelForCausalLM

model = AutoModelForCausalLM.from_pretrained(
    "TinyPixel/Llama-2-7B-bf16-sharded",
    load_in_8bit=True,
    device_map='auto',
)

tokenizer = AutoTokenizer.from_pretrained("TinyPixel/Llama-2-7B-bf16-sharded")

config.json:   0%|          | 0.00/626 [00:00<?, ?B/s]

pytorch_model.bin.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

pytorch_model-00001-of-00014.bin:   0%|          | 0.00/981M [00:00<?, ?B/s]

pytorch_model-00002-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

pytorch_model-00003-of-00014.bin:   0%|          | 0.00/967M [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/132 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/676 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/411 [00:00<?, ?B/s]

### Freezing the original weights


In [5]:
for param in model.parameters():
  param.requires_grad = False  # freeze the model - train adapters later
  if param.ndim == 1:
    # cast the small parameters (e.g. layernorm) to fp32 for stability
    param.data = param.data.to(torch.float32)

model.gradient_checkpointing_enable()  # reduce number of stored activations
model.enable_input_require_grads()

class CastOutputToFloat(nn.Sequential):
  def forward(self, x): return super().forward(x).to(torch.float32)
model.lm_head = CastOutputToFloat(model.lm_head)

### Setting up the LoRa Adapters

In [6]:
def print_trainable_parameters(model):
    """
    Prints the number of trainable parameters in the model.
    """
    trainable_params = 0
    all_param = 0
    for _, param in model.named_parameters():
        all_param += param.numel()
        if param.requires_grad:
            trainable_params += param.numel()
    print(
        f"trainable params: {trainable_params} || all params: {all_param} || trainable%: {100 * trainable_params / all_param}"
    )

In [9]:
from peft import LoraConfig, get_peft_model

config = LoraConfig(
    r=32, #attention heads
    lora_alpha=64, #alpha scaling
    # target_modules=["q_proj", "v_proj"], #if you know the
    lora_dropout=0.25,
    bias="none",
    task_type="CAUSAL_LM" # set this for CLM or Seq2Seq
)

model = get_peft_model(model, config)
print_trainable_parameters(model)

trainable params: 16777216 || all params: 6755192832 || trainable%: 0.24836028248556738


## Data

In [10]:
import transformers
from datasets import load_dataset
data = load_dataset("nihiluis/financial-advisor-100")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/100 [00:00<?, ? examples/s]

In [12]:
def merge_columns(example):
    example["prediction"] = f"### Instruction: {example['question']} \n\n### Response: {example['question']}"
    return example

data['train'] = data['train'].map(merge_columns)
data['train']["prediction"][:5]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

['### Instruction: I grew up as an only child with my dad working and my mom stayed home.  We never have all of the fanciest things or unnecessary things, but it also didn\'t seem like we missed out on anything.  I observed and learned that they just didn\'t spend money they didn\'t have, saved up, and invested wisely.  They never had a financial advisor and just accumulated a bunch of Microsoft stock since the 90\'s along with whatever my dad researched and never needed to sell it because they still don\'t really buy anything or go anywhere, which is kind of sad but they have plenty if ever needed.  I sort of followed in those footsteps and now at 45 and married with 2 kids, we tend to save a lot, don\'t buy a bunch of unnecessary things, and don\'t spend money we don\'t have.  Luckily they invested some money in Microsoft for me back then and I still have several hundred thousand worth, even after using some to buy a house, along with my 401k, etc.  The reason I am bringing this is u

In [13]:
data['train'][0]

{'id': '1006ury',
 'question': 'I grew up as an only child with my dad working and my mom stayed home.  We never have all of the fanciest things or unnecessary things, but it also didn\'t seem like we missed out on anything.  I observed and learned that they just didn\'t spend money they didn\'t have, saved up, and invested wisely.  They never had a financial advisor and just accumulated a bunch of Microsoft stock since the 90\'s along with whatever my dad researched and never needed to sell it because they still don\'t really buy anything or go anywhere, which is kind of sad but they have plenty if ever needed.  I sort of followed in those footsteps and now at 45 and married with 2 kids, we tend to save a lot, don\'t buy a bunch of unnecessary things, and don\'t spend money we don\'t have.  Luckily they invested some money in Microsoft for me back then and I still have several hundred thousand worth, even after using some to buy a house, along with my 401k, etc.  The reason I am bring

In [14]:
data = data.map(lambda samples: tokenizer(samples['prediction']), batched=True)

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [15]:
data

DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'text', 'prediction', 'input_ids', 'attention_mask'],
        num_rows: 100
    })
})

### Training

In [19]:

trainer = transformers.Trainer(
    model=model,
    train_dataset=data['train'],
    args=transformers.TrainingArguments(
        per_device_train_batch_size=4,
        gradient_accumulation_steps=4,
        warmup_steps=100,
        max_steps=200,
        learning_rate=2e-4,
        fp16=True,
        logging_steps=1,
        output_dir='outputs'
    ),
    data_collator=transformers.DataCollatorForLanguageModeling(tokenizer, mlm=False)
)
tokenizer.pad_token = tokenizer.eos_token
model.config.use_cache = False  # silence the warnings. Please re-enable for inference!
trainer.train()

OutOfMemoryError: ignored